# Предсказание атрибутов ребер с использованием графовых нейронных сетей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Курс "Машинное обучение на графах", Лекции 4-5 "Графовые нейронные сети"

Документация:
* https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
* https://docs.dgl.ai/generated/dgl.from_networkx.html
* https://pytorch.org/docs/stable/generated/torch.bernoulli.html
* https://docs.dgl.ai/generated/dgl.remove_edges.html
* https://docs.dgl.ai/guide/training-edge.html
* https://www.sbert.net/

## Вопросы для совместного обсуждения

1\. Обсудите основные шаги для решения задачи предсказания атрибутов узлов при помощи графовых нейронных сетей.

In [ ]:
!pip install dgl

In [ ]:
!pip install torch==2.1.2

In [ ]:
import dgl
import torch as th
import dgl.nn as gnn

In [ ]:
dset = dgl.data.KarateClubDataset()
g = dset[0]
g.ndata["feats"] = th.eye(g.num_nodes())
g.edata["labels"] = th.rand(g.num_edges())

In [ ]:
import dgl.nn as gnn
import torch.nn as nn


class ModelGNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = gnn.GraphConv(34, 16)

  def forward(self, g, x):
    return self.conv1(g, x)

In [ ]:
model = ModelGNN()
h = model(g, g.ndata["feats"])

In [ ]:
h.shape

torch.Size([34, 16])

In [ ]:
g.num_edges()

156

In [ ]:
import dgl.function as fn
import torch.nn as nn

class DotProductPredictor(nn.Module):
    def forward(self, graph, h):
        # h contains the node representations computed from the GNN defined
        # in the node classification section (Section 5.1).
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return graph.edata['score']

In [ ]:
predictor = DotProductPredictor()
preds = predictor(g, h)
preds.shape

torch.Size([156, 1])

In [ ]:
import torch

class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        # h contains the node representations computed from the GNN defined
        # in the node classification section (Section 5.1).
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [ ]:
predictor = MLPPredictor(in_features=16, out_classes=7)
preds = predictor(g, h)
preds.shape

torch.Size([156, 7])

In [ ]:
g.ndata["pos_id"] = th.randint(0, 10, size=(g.num_nodes(), ))

In [ ]:
emb = nn.Embedding(num_embeddings=10, embedding_dim=32)
emb(g.ndata["pos_id"]).shape

torch.Size([34, 32])

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
from  sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("cointegrated/rubert-tiny2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
texts = ["Текст #{i}" for i in range(10)]
texts_emb = model.encode(texts, convert_to_tensor=True)

In [ ]:
texts_emb.shape

torch.Size([10, 312])

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. В файлах каталога `rwn` находятся данных о синсетах русскоязычного WordNet и связях между ними. Создайте `nx.DiGraph`, считав данные из этих файлов. Узлами графа являются синсеты из файлов `synsets.[AVN].xml`. В качестве атрибутов узлов сохраните `name` (атрибут `ruthes_name`) и `part_of_speech`. В качестве идентификаторов узлов используйте атрибут `id`. Ребрами графа являются отношения из файлов `synset_relations.[AVN].xml`. В качестве атрибутов ребер сохраните `relation_name` (атрибут `name`). Выведите на экран количество узлов и ребер в полученном графе. Выведите на экраны атрибуты узла `A7417`. Выведите на экран атрибуты ребра между `A1` и `A7417`.

- [ ] Проверено на семинаре

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
file_path = '/content/drive/My Drive/rwn.zip'
folder = '/content/'
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(folder)

In [ ]:
import os
import xml.etree.ElementTree as ET
import networkx as nx

In [ ]:
def parse_synsets(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    synsets = {}
    for synset in root.findall('synset'):
        synset_id = synset.get('id')
        ruthes_name = synset.get('ruthes_name')
        part_of_speech = synset.get('part_of_speech')
        synsets[synset_id] = {
            'name': ruthes_name,
            'part_of_speech': part_of_speech
        }
    return synsets

def parse_synset_relations(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    relations = []
    for relation in root.findall('relation'):
        parent_id = relation.get('parent_id')
        child_id = relation.get('child_id')
        relation_name = relation.get('name')
        relations.append((parent_id, child_id, {'relation_name': relation_name}))
    return relations

def build_graph(synsets_files, relations_files):
    G = nx.DiGraph()
    for file in synsets_files:
        synsets = parse_synsets(file)
        for synset_id, attributes in synsets.items():
            G.add_node(synset_id, **attributes)

    for file in relations_files:
        relations = parse_synset_relations(file)
        G.add_edges_from(relations)

    return G


In [ ]:
synsets_files = ['rwn/synsets.A.xml', 'rwn/synsets.V.xml', 'rwn/synsets.N.xml']
relations_files = ['rwn/synset_relations.A.xml', 'rwn/synset_relations.V.xml', 'rwn/synset_relations.N.xml']

In [ ]:
G = build_graph(synsets_files, relations_files)

In [ ]:
G.number_of_nodes()


49492

In [ ]:
G.number_of_edges()

221618

In [ ]:
G.nodes['A7417']

{'name': 'ПОДПИСАТЬСЯ НА УСЛУГУ', 'part_of_speech': 'Adj'}

In [ ]:
G['A1']['A7417']

{'relation_name': 'hypernym'}

<p class="task" id="2"></p>

2\. Закодируйте названия отношений на ребрах при помощи `LabelEncoder` из `sklearn` и сохраните результат в виде атрибута `relation_id` на ребрах. Закодируйте часть речи синсетов при помощи `LabelEncoder` из `sklearn` и сохраните результат в виде атрибута `pos_id` на узлах. Используя пакет `sentence_transformers`, получите векторное представление названия синсетов и сохраните результат в виде атрибута `name_embedding` на узлах.

На основе созданного `nx.DiGraph` создайте `DGLGraph` при помощи функции `dgl.from_networkx` с сохранением всех числовых атрибутов. Добавьте на ребра `DGLGraph` булеву маску `train_mask` (80% True). Добавьте на ребра `DGLGraph` булеву маску `test_mask` (инвертированный `train_mask`).

Выведите на экран названия всех атрибутов узлов и ребер, а также размерности соответствующих им тензоров.

Создайте версию графа `g_train` без ребер из тестовой выборки. Выведите количество узлов и ребер в `g_train`.
- [ ] Проверено на семинаре

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
import numpy as np


In [ ]:
relation_encoder = LabelEncoder()
relation_names = [data['relation_name'] for _, _, data in G.edges(data=True)]
relation_ids = relation_encoder.fit_transform(relation_names)


In [ ]:
for i, (u, v, data) in enumerate(G.edges(data=True)):
    G[u][v]['relation_id'] = relation_ids[i]

In [ ]:
pos_encoder = LabelEncoder()
pos_labels = [data['part_of_speech'] for _, data in G.nodes(data=True)]
pos_ids = pos_encoder.fit_transform(pos_labels)


In [ ]:
for i, (node, data) in enumerate(G.nodes(data=True)):
    G.nodes[node]['pos_id'] = pos_ids[i]

In [ ]:
model = SentenceTransformer("cointegrated/rubert-tiny2")

In [ ]:
names = [data['name'] for _, data in G.nodes(data=True)]
name_embeddings = model.encode(names)

In [ ]:
for i, (node, data) in enumerate(G.nodes(data=True)):
    G.nodes[node]['name_embedding'] = name_embeddings[i]

In [ ]:
dgl_graph = dgl.from_networkx(G, node_attrs=['pos_id', 'name_embedding'], edge_attrs=['relation_id'])

In [ ]:
num_edges = dgl_graph.number_of_edges()
train_mask = th.zeros(num_edges, dtype=bool)
train_size = int(0.8 * num_edges)
train_mask[:train_size] = True
test_mask = ~train_mask

In [ ]:
dgl_graph.edata['train_mask'] = train_mask
dgl_graph.edata['test_mask'] = test_mask

In [ ]:
dgl_graph.ndata.keys()

dict_keys(['pos_id', 'name_embedding'])

In [ ]:
dgl_graph.edata.keys()

dict_keys(['relation_id', 'train_mask', 'test_mask'])

In [ ]:
dgl_graph.ndata['pos_id'].shape

torch.Size([49492])

In [ ]:
dgl_graph.ndata['name_embedding'].shape

torch.Size([49492, 312])

In [ ]:
dgl_graph.edata['relation_id'].shape

torch.Size([221618])

In [ ]:
test_edge_indices = (~train_mask).nonzero(as_tuple=False).squeeze()
g_train = dgl.remove_edges(dgl_graph, test_edge_indices)

In [ ]:
g_train.number_of_nodes()

49492

In [ ]:
g_train.number_of_edges()

177294

In [ ]:
177294/221618


0.7999981950924564

<p class="task" id="3"></p>

3\. Решите задачу предсказания классов ребер, используя два слоя `SAGEConv` и `MLPPredictor` для получения прогнозов на ребрах. В качестве признаков узлов используйте только эмбеддинги наименования синсетов. Во время обучения используйте граф `g_train`, не содержащий ребра из тестовой выборки.

Рассчитайте значение метрики accuracy на обучающем множестве и на тестовом множестве для обученной модели. Для расчета метрик используйте исходный граф (до удаления ребер).

- [ ] Проверено на семинаре

In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [ ]:
class SAGEModel(nn.Module):
    def __init__(self, in_feats, hidden_feats):
        super(SAGEModel, self).__init__()
        self.sage1 = gnn.SAGEConv(in_feats, hidden_feats, aggregator_type='mean')
        self.sage2 = gnn.SAGEConv(hidden_feats, hidden_feats, aggregator_type='mean')

    def forward(self, g, x):
        h = self.sage1(g, x)
        h = torch.relu(h)
        h = self.sage2(g, h)
        return h

In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super(MLPPredictor, self).__init__()
        self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [ ]:
in_feats = dgl_graph.ndata['name_embedding'].shape[1]
hidden_feats = 16
out_classes = len(relation_encoder.classes_)

In [ ]:
sage_model = SAGEModel(in_feats, hidden_feats)
mlp_predictor = MLPPredictor(hidden_feats, out_classes)

In [ ]:
optimizer = torch.optim.Adam(list(sage_model.parameters()) + list(mlp_predictor.parameters()))

inputs = g_train.ndata['name_embedding']
labels = g_train.edata['relation_id']

In [ ]:
for epoch in range(200):
    h = sage_model(g_train, inputs)
    logits = mlp_predictor(g_train, h)
    loss = F.cross_entropy(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()




In [ ]:
with torch.no_grad():
    h = sage_model(g_train, inputs)
    logits = mlp_predictor(g_train, h)
    train_acc = accuracy_score(labels.numpy(), logits.argmax(1).numpy())
    test_acc = accuracy_score(labels.numpy(), logits.argmax(1).numpy())

print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)


Train Accuracy: 0.4935643620201473
Test Accuracy: 0.4935643620201473


<p class="task" id="4"></p>

4\. Решите предыдущую задачу, задействовав информацию о частях речи синсетов в процессе обучения модели. Для этого создайте дополнительный слой `nn.Embedding`, который будет использоваться для получения эмбеддингов частей речи (атрибут `pos_id`). Для получения вектора признаков для каждого узла объедините эмбеддинг названия синсета и эмбеддинг части речи в один длинный вектор при помощи `torch.cat`.

После завершения обучения рассчитайте accuracy на обучающем и тестовом множестве. Сделайте выводы.

- [ ] Проверено на семинаре

In [188]:
num_pos = len(pos_encoder.classes_)
pos_embedding = nn.Embedding(num_pos, in_feats)

In [189]:
inputs = torch.cat([name_embeddings, pos_embedding(pos_ids)], dim=1)

In [190]:
name_embeddings = g_train.ndata['name_embedding']

In [191]:
sage_model = SAGEModel(in_feats * 2, hidden_feats)
mlp_predictor = MLPPredictor(hidden_feats, out_classes)
optimizer = torch.optim.Adam(list(sage_model.parameters()) + list(mlp_predictor.parameters()))


In [192]:
for epoch in range(200):
    pos_ids = g_train.ndata['pos_id']
    inputs = torch.cat([name_embeddings, pos_embedding(pos_ids)], dim=1)

    h = sage_model(g_train, inputs)
    logits = mlp_predictor(g_train, h)
    loss = F.cross_entropy(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [193]:
with torch.no_grad():
    pos_ids = g_train.ndata['pos_id']
    inputs = torch.cat([name_embeddings, pos_embedding(pos_ids)], dim=1)

    h = sage_model(g_train, inputs)
    logits = mlp_predictor(g_train, h)
    train_acc = accuracy_score(labels.numpy(), logits.argmax(1).numpy())
    test_acc = accuracy_score(labels.numpy(), logits.argmax(1).numpy())

print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)


Train Accuracy: 0.7088339142892597
Test Accuracy: 0.7088339142892597


In [193]:
# точность сильно улучшилась